# Models Analysis
Three models are developed and compared to determine which is the best one in terms of company strategy forecasting accuract.<br>
The tested models are LSTM, ARIMA and SVR.

In [ ]:
import sys
sys.path.append("..")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.lstm import Correlation
from matplotlib import rcParams 
from sklearn.metrics import roc_curve

from src.lstm import Preprocess
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Dropout, Bidirectional, TimeDistributed
from keras.optimizers import Adam
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
import statsmodels.graphics.tsaplots as tplot
from statsmodels.tsa.arima_model import ARIMA
import seaborn as sns

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from svr import Svr

from src.arima import Arima
from influxdb import InfluxDBClient
from dateutil import relativedelta


rcParams.update({'font.size': 11})
rcParams.update({'figure.autolayout': True})

## 1) LSTM
### 1.1) Correlation Analysis

The first step of the LSTM ananlysis is to detect which features are more informative.<br>
Each MGP feature is correlated with the target variable and the ones with a correlation greater than 40% with the target one is saved, all the others are dropped.

In [ ]:
SPLIT = .7
TIMESTEPS = 7
MIN_CORR = .4

In [ ]:
p = Correlation(MIN_CORR)
data, label = p.loadData()

corr = p.selectFeatures(data, label)
corr = corr.sort_values('Target', ascending=False)
corr = corr.drop('Target')
corr = corr.rename(
    {
        'DLY_QTY':'y(t)',
        'MGP_CNOR_Prezzo':'C-North Price',
        'MGP_AUST_Prezzo':'AT Price',
        'MGP_BSP_Prezzo':'SVN Coupling Price',
        'MGP_FRAN_Prezzo':'FR Price',
        'MGP_NORD_Prezzo':'North Price',
        'MGP_SLOV_Prezzo':'SVN Coupling Price',
        'MGP_SVIZ_Prezzo':'CH Coupling Price',
        'MGP_XAUS_Prezzo':'AT Coupling Price',
        'MGP_XFRA_Prezzo':'FR Coupling Price',
        'MGP_PUN_Prezzo':'National Price',
        'MGP_NAT_Prezzo':'NAT Price',
        'MGP_CSUD_Prezzo':'C-South Price',
        'MGP_COAC_Prezzo':'Corsica Price',
        'MGP_SARD_Prezzo':'Sardinia Coupling Price'
    }
)
corr['Target']*=100

In [ ]:
plt.figure()
plt.axhline(y=50, color='k', linestyle='-.', linewidth=1)
plt.axhline(y=45, color='k', linestyle='-.', linewidth=1)
plt.axhline(y=40, color='k', linestyle='-.', linewidth=1)
plt.stem(corr.index,corr,  bottom=-2)
plt.yticks([40,45,50,70,100])
plt.ylim(30)
plt.xlabel('Features')
plt.ylabel('Correlation [%]')
plt.xticks(rotation=90)
plt.savefig('../fig/featuresCorrelation.png', transparent=True)

### 1.2) Model Training
Now the model is developed. Two scenarios are investigated: <br>
1. Next day forecasting <br>
2. 8 days forecasting. In this second case the missing 7 days are forecasted and appended to the original dataset by considering them as observations<br>
<br>
Since from the correlation analysis there is only one feature with correlation greater than 50%, which is the time-shifted target variable, the LSTM model is developed to work only with it.

In [ ]:
SPLIT = .7
TIMESTEPS = 7
MIN_CORR = .5

In [ ]:
p = Preprocess(MIN_CORR)
data , label= p.loadData()
label = label.reshape(-1, 1)

Create the X and y datasets

In [ ]:
X = p.reshapeData(data)
y = p.reshapeData(label)
y = y.reshape(y.shape[0], y.shape[1])

Scale the datasets

In [ ]:
x_train, y_train, x_test, y_test = p.scaleData(X, y)

Initialize the model

In [ ]:
model = Sequential()

model.add(
    LSTM(
        200, 
        input_shape=(x_train.shape[1], x_train.shape[2]), 
        return_sequences=True, 
        unroll=True
    )
)
model.add(Dropout(.2))

model.add(
    Bidirectional(
        LSTM(
            80, 
            input_shape=(x_train.shape[1], x_train.shape[2]),
            unroll=True,
            return_sequences=True, 
        )
    )
)
model.add(Dropout(.2))

model.add(TimeDistributed(Dense(1)))

opt = Adam(learning_rate=0.0001)

model.compile(loss='mse', optimizer=opt , metrics = ['mae', 'mape'])

history = model.fit(
    x_train,
    y_train,
    epochs=1000, 
    batch_size=20, 
    validation_data=(x_test,y_test), 
    shuffle=False,
)

model.save('../models/lstm.h5') 

In [ ]:
Plot the loss function trend

In [ ]:
plt.plot(history.history['loss'], label='Training', linewidth=2.5)
plt.plot(
    history.history['val_loss'], label='Test', linewidth=2.5, linestyle='-.', color = 'r'
)
plt.legend()
plt.grid(linestyle = '--')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.savefig('../fig/loss.png', transparent=True)

### Model Validation
The entire dataset is forecasted and analyzed.

#### Next Day Validation

In [ ]:
reprocess(MIN_CORR)
data , label= p.loadData()
label = label.reshape(-1, 1)

X = p.reshapeData(data)
y = p.reshapeData(label)
y = y.reshape(y.shape[0], y.shape[1])

scaled = p.scaleData(X)
model = load_model('../models/lstm.h5')

y_hat = []
for i in range(scaled.shape[0]):
    y_hat.append(model.predict(scaled[i].reshape(1,scaled.shape[1], scaled.shape[2]))[0])

y_hat = np.asarray(y_hat)
y_hat = p.descaleData(y_hat)

Save the results

In [ ]:
to_save = {'y':y[:,-1],'y_hat':y_hat[:,-1]}
pd.DataFrame(to_save).to_csv('../data/lstm1days.csv')

#### 8 Days Validation

In [ ]:
p = Preprocess(MIN_CORR)
original_data ,_= p.loadData()

y_hat = []
y = []
model = load_model('../models/lstm.h5')
for j in range(original_data.shape[0]):
    # Emulate the next day
    temp_data = original_data.copy()
    try:
        for i in range(8):
            pred = []
            # Load TIMESTEPS samples
            data = temp_data.iloc[i+j:i+j+TIMESTEPS].copy()
            # Create one single sample of 3D data
            X = p.reshapeData(data)
            X = X[0,:,:]
            X = X.reshape(1,X.shape[0],X.shape[1])
            # Scale Data
            scaled = p.scaleData(X)
            # Predict the next missing sample of Public Offers
            pred.append(model.predict(scaled.reshape(1,scaled.shape[1], scaled.shape[2]))[0])
            missing_off = p.descaleData(np.asarray(pred))[0,-1]
            # If i is 7, the missing week has been
            # forecasted, so the original sample is put in y, whereas
            # the forecasted one is the target one (t+1)
            if i == 7:
                y.append(original_data.iloc[i+j+TIMESTEPS]['DLY_QTY'])
                y_hat.append(missing_off)
            # Replace the real data retrieved for training purpose with
            # the forecasted sample.
            temp_data.iloc[i+j+TIMESTEPS]['DLY_QTY'] = missing_off
    except:
        pass

Save the result

In [ ]:
to_save = {'y':y,'y_hat':y_hat}
pd.DataFrame(to_save).to_csv('../data/lstm8days.csv')

## 2) ARIMA
The first step consists on analyzing the timeseries stationarity, since the ARIMA model can be applied only to stationary timeseries. Both a analytical method (Augmented Dickey Fuller test) and a graphical one based on the ACF and PACF analysis are used.<br>
Then, in case of stationarity, a grid search is performed to determine the best ARIMA hyperparameters.

In [ ]:
def dickeyFuller(data, model):
    cutoff=0.00001
    # Perform Dickey-Fuller test:
    print(
        f'Dickey-Fuller Test {model}:'
    )
    p_val = adfuller(data['DLY_QTY'].dropna(), autolag='AIC')[1]
    if p_val < cutoff:
        print(f'\tp-value = {round(p_val,2)}. Stationary.')
    else:
        print(f'\tp-value = {round(p_val,2)}. Non-stationary.')


def statTest(data):
    # 0 order differentiation
    rolling_mean = data.rolling(window = 24).mean()
    rolling_std = data.rolling(window = 24).std()
    dickeyFuller(data, '0d')
    # 1 order differentiation
    rolling_mean_diff = (
        data
        .diff()
        .rolling(window = 24)
        .mean()
    )
    rolling_std_diff = (
        data
        .diff()
        .rolling(window = 24)
        .std()
    )
    dickeyFuller(data.diff(), '1d')

    # Plot the results
    plt.figure()
    plt.plot(data, label='Original Timeseries')
    plt.plot(rolling_mean, label='Rolling mean', color='r')
    plt.plot(rolling_std, label='Rolling std', color='k')
    plt.xlabel('Date')
    plt.ylabel('Offered Daily Quantity')
    plt.xticks(rotation=90)
    plt.grid()
    plt.legend(ncol=3, loc='upper center', bbox_to_anchor=(0.5, 1.30))
    plt.savefig('../fig/rollingD0.png', transparent=True)
    plt.close()

    # Plot the results
    plt.figure()
    plt.plot(data.diff(), label='Differenced Timeseries')
    plt.plot(rolling_mean_diff, label='Rolling mean', color='r')
    plt.plot(rolling_std_diff, label='Rolling std', color='k')
    plt.xlabel('Date')
    plt.ylabel('Offered Daily Quantity')
    plt.xticks(rotation=90)
    plt.grid()
    plt.legend(ncol=3, loc='upper center', bbox_to_anchor=(0.5, 1.30))
    plt.savefig('../fig/rollingD1.png', transparent=True)
    plt.close()

### 2.1) Stationarity Test

PLIM = 6
QLIM = 4

WINDOW = 60

In [ ]:
off = (
    pd
    .read_csv('../data/bid.csv', index_col='Unnamed: 0')
    .drop(
        columns = [
            'DLY_AWD_QTY',
            'TYPE', 
            'DLY_PRICE', 
            'DLY_AWD_PRICE'
        ]
    )
    .dropna()
)
df = off 
df = df.set_index(pd.DatetimeIndex(off.index))
df = df.resample('D').sum()

Perform the Dickey Fuller test and check if the 1st order differentiation improves the performances

In [ ]:
statTest(df)

Perform the ACF and PACF analysis

In [ ]:
data = df
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(15,10))
tplot.plot_acf(
    data, 
    ax[0],
    lags=40, 
    markersize=0,
    title='Original'
)
tplot.plot_acf(
    data.diff().dropna(), 
    ax[1],
    lags=40, 
    markersize=0,
    title='1st Order Differentiation'
)

plt.savefig('../fig/acf.png', transparent=True)
plt.close()

In [ ]:
data = df
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(15,10))
tplot.plot_pacf(
    data, 
    ax[0],
    lags=40, 
    markersize=0,
    title='Original'
)
tplot.plot_pacf(
    data.diff().dropna(), 
    ax[1],
    lags=40, 
    markersize=0,
    title='1st Order Differentiation'
)

plt.savefig('../fig/pacf.png', transparent=True)
plt.close()

### 2.2) Grid Search

#### 2.2.1) Next Day Forecasting

In [ ]:
h = 1

X = df.values
X = X[1:]

p_val = np.arange(0,PLIM)
q_val = np.arange(0,QLIM)

mse = np.zeros((len(p_val), len(q_val)))
mape = np.zeros((len(p_val), len(q_val)))
for p in p_val:
    for q in q_val:
        y = []
        y_hat = []
        print(f'ARIMA({p},1,{q})')
        for i in range(X.shape[0]):
            try:
                train = X[i:i+WINDOW]

                model = ARIMA(
                    train, 
                    order=(p,1,q),
                )
                model_fit = model.fit(maxiter=200, disp=0, method='css')
                y.append(X[i+WINDOW-1])
                y_hat.append(model_fit.forecast(h)[0][-1])
            except:
                pass
        mse[p_val[p]][q_val[q]] = mean_squared_error(y, y_hat)

Getting the MSE Heatmap

In [ ]:
plt.figure(figsize=(6,8))
ax = sns.heatmap(mse, annot=True, fmt='.1f', cmap='YlGnBu', square=True, cbar_kws={'label':'MSE'}, vmin=0)
ax.set_ylim(PLIM,0)
ax.set_xlim(QLIM,0)
ax.invert_yaxis()
ax.invert_xaxis()
ax.set_ylabel('Autoregressive Order (p)')
ax.set_xlabel('Moving Average Order (q)')
plt.savefig('../fig/heatmap1day.png', transparent=True)
plt.close()

#### 2.2.2) 8 Day Forecasting

In [ ]:
h = 8

X = df.values
X = X[1:]

p_val = np.arange(0,PLIM)
q_val = np.arange(0,QLIM)

mse = np.zeros((len(p_val), len(q_val)))
mape = np.zeros((len(p_val), len(q_val)))
for p in p_val:
    for q in q_val:
        y = []
        y_hat = []
        print(f'ARIMA({p},1,{q})')
        for i in range(X.shape[0]):
            try:
                train = X[i:i+WINDOW]

                model = ARIMA(
                    train, 
                    order=(p,1,q),
                )
                model_fit = model.fit(maxiter=200, disp=0, method='css')
                y.append(X[i+WINDOW-1])
                y_hat.append(model_fit.forecast(h)[0][-1])
            except:
                pass
        mse[p_val[p]][q_val[q]] = mean_squared_error(y, y_hat)

Getting the MSE Heatmap

In [ ]:
plt.figure(figsize=(6,8))
ax = sns.heatmap(mse, annot=True, fmt='.1f', cmap='YlGnBu', square=True, cbar_kws={'label':'MSE'}, vmin=0)
ax.set_ylim(PLIM,0)
ax.set_xlim(QLIM,0)
ax.invert_yaxis()
ax.invert_xaxis()
ax.set_ylabel('Autoregressive Order (p)')
ax.set_xlabel('Moving Average Order (q)')
plt.savefig('fig/heatmap8day.png', transparent=True)
plt.close()

### 2.3) Model Implementation
Since the ARIMA(0,1,0) leads to the best performances, the final model has been implemented and then the entire dataset is investigated to check the forecasting accuracy.<br>
Two scenarios are investigated: <br>
1. Next day forecasting <br>
2. 8 days forecasting <br>
The observation window size is fixed to 60 past days

In [ ]:
WINDOW = 60

Load data and perform a daily resampling.

In [ ]:
off = (
    pd
    .read_csv('../data/bid.csv', index_col='Unnamed: 0')
    .drop(
        columns = [
            'DLY_AWD_QTY',
            'TYPE', 
            'DLY_PRICE', 
            'DLY_AWD_PRICE'
        ]
    )
    .dropna()
)
df = off 
df = df.set_index(pd.DatetimeIndex(off.index))
df = df.resample('D').sum()

#### 2.3.1) Next Day Validation
The horizon is set equal to 1 and all the dataset is iteratively<br>
analyzed by making the window sliding from one day to the next one.

In [ ]:
h = 1

X = df.values
X = X[1:]

y = []
y_hat = []

for i in range(X.shape[0]):
    try:
        train = np.copy(X[i:i+WINDOW])

        model = ARIMA(
            train, 
            order=(0,1,0),
        )
        model_fit = model.fit(maxiter=200, disp=0, method='css')
        y.append(X[i+WINDOW-1])
        y_hat.append(model_fit.forecast(h)[0][-1])
    except:
        pass

Save the result in a .csv file which will be used <br>
in the models comparison phase.

In [ ]:
to_save = {'y':y, 'y_hat':y_hat}
pd.DataFrame(to_save).to_csv('../data/arima1days.csv')

#### 2.3.2) 8 Days Validation
The horizon is set equal to 8, this means tht at each timestep, <br>
the next eighth day is forecasted. All the dataset is iteratively<br>
analyzed by making the window sliding from one day to the next one.

In [ ]:
h = 8

X = df.values
X = X[1:]

y = []
y_hat = []

for i in range(X.shape[0]):
    try:
        train = np.copy(X[i:i+WINDOW])

        model = ARIMA(
            train, 
            order=(0,1,0),
        )
        model_fit = model.fit(maxiter=200, disp=0, method='css')
        y.append(X[i+WINDOW-1])
        y_hat.append(model_fit.forecast(h)[0][-1])
    except:
        pass

Save the result in a .csv file which will be used <br>
in the models comparison phase.

In [ ]:
to_save = {'y':y,'y_hat':y_hat}
pd.DataFrame(to_save).to_csv('../data/arima8days.csv')

## 3) SVR
A grid search is performed to detect the best observation window. Two scenarios are investigated: <br>
1. Next day forecasting <br>
2. 8 days forecasting. In this second case the missing 7 days are forecasted and appended to the original dataset by considering them as observations<br>


In [ ]:
rcParams.update({'font.size': 14})

MIN_CORR = .1
N_WINDOW = 62

### 3.1) Grid Search

#### 3.1.1) Next Day Forecasting

In [ ]:
p = Svr()
original_data, _= p.loadData()
N=1
MSE = []
for WINDOW in np.arange(2, N_WINDOW):
    print(f'\tWindow size:{WINDOW}')
    y = []
    y_hat = []
    for j in range(original_data.shape[0]):
        # Emulate the next day
        temp_data = original_data.copy()
        try:
            for i in range(N):
                pred = []
                # Load the last available sample
                X = temp_data.values[j+i:i+j+WINDOW+1].reshape(WINDOW+1,-1)
                target = temp_data.values[i+j+1:i+j+WINDOW+2, -1].reshape(WINDOW+1,-1)
                # Scale Data
                x_train, y_train, x_test, y_test = p.scaleData(WINDOW, X, target)
                model = SVR(kernel='rbf',gamma='auto',C=1.0,epsilon=0.1)
                model.fit(x_train, y_train[:,0])
                # Predict the next missing sample of Public Offers
                pred.append(model.predict(x_test))
                missing_off = p.descaleData(np.asarray(pred))[0,0]
                # If i is 7, the missing week has been
                # forecasted, so the original sample is put in y, whereas
                # the forecasted one is the target one (t+1)
                if i == N-1:
                    y.append(p.descaleData(np.asarray(y_test))[0,0])
                    y_hat.append(missing_off)
                # Replace the real data retrieved for training purpose with
                # the forecasted sample.
                temp_data.iloc[i+j+1]['DLY_QTY'] = missing_off
        except:
            pass
    # Compute metrics
    mse = mean_squared_error(y, y_hat)
    MSE.append(mse)

In [ ]:
fig = plt.figure()
plt.xlabel('Window Size')
plt.ylabel('MSE')
plt.plot(np.arange(2, N_WINDOW), MSE, color='b', linestyle='-.',label='MSE')
plt.grid(linestyle='-.')
plt.savefig('../fig/svrSearch1d.png', transparent=True)
plt.close()

#### 8 Days Forecasting

In [ ]:
p = Svr()
original_data, _= p.loadData()
N=8
MSE = []
for WINDOW in np.arange(2, N_WINDOW):
    print(f'\tWindow size:{WINDOW}')
    y = []
    y_hat = []
    for j in range(original_data.shape[0]):
        # Emulate the next day
        temp_data = original_data.copy()
        try:
            for i in range(N):
                pred = []
                # Load the last available sample
                X = temp_data.values[j+i:i+j+WINDOW+1].reshape(WINDOW+1,-1)
                target = temp_data.values[i+j+1:i+j+WINDOW+2, -1].reshape(WINDOW+1,-1)
                # Scale Data
                x_train, y_train, x_test, y_test = p.scaleData(WINDOW, X, target)
                model = SVR(kernel='rbf',gamma='auto',C=1.0,epsilon=0.1)
                model.fit(x_train, y_train[:,0])
                # Predict the next missing sample of Public Offers
                pred.append(model.predict(x_test))
                missing_off = p.descaleData(np.asarray(pred))[0,0]
                # If i is 7, the missing week has been
                # forecasted, so the original sample is put in y, whereas
                # the forecasted one is the target one (t+1)
                if i == N-1:
                    y.append(p.descaleData(np.asarray(y_test))[0,0])
                    y_hat.append(missing_off)
                # Replace the real data retrieved for training purpose with
                # the forecasted sample.
                temp_data.iloc[i+j+1]['DLY_QTY'] = missing_off
        except:
            pass
    # Compute metrics
    mse = mean_squared_error(y, y_hat)
    MSE.append(mse)

In [ ]:
fig = plt.figure()
plt.grid(linestyle='-.')
plt.xlabel('Window Size')
plt.ylabel('MSE')
plt.plot(np.arange(2, N_WINDOW), MSE, color='b', linestyle='-.',label='MSE')
plt.savefig('../fig/svrSearch8d.png', transparent=True)
plt.close()

### 3.2) Model Validation
The entire dataset is investigated

In [ ]:
rcParams.update({'font.size': 11})
TIMESTEPS = 7
WINDOW = 7
MIN_CORR = .1

#### Next Day Validation

In [ ]:
p = Svr()
original_data, _= p.loadData()

N=1
y = []
y_hat = []
for j in range(original_data.shape[0]):
    # Emulate the next day
    temp_data = original_data.copy()
    try:
        for i in range(N):
            pred = []
            # Load the last available sample
            X = temp_data.values[j+i:i+j+WINDOW+1].reshape(WINDOW+1,-1)
            target = temp_data.values[i+j+1:i+j+WINDOW+2, -1].reshape(WINDOW+1,-1)
            # Scale Data
            x_train, y_train, x_test, y_test = p.scaleData(WINDOW, X, target)
            model = SVR(kernel='rbf',gamma='auto',C=1.0,epsilon=0.1)
            model.fit(x_train, y_train[:,0])
            # Predict the next missing sample of Public Offers
            pred.append(model.predict(x_test))
            missing_off = p.descaleData(np.asarray(pred))[0,0]
            # If i is 7, the missing week has been
            # forecasted, so the original sample is put in y, whereas
            # the forecasted one is the target one (t+1)
            if i == N-1:
                y.append(p.descaleData(np.asarray(y_test))[0,0])
                y_hat.append(missing_off)
            # Replace the real data retrieved for training purpose with
            # the forecasted sample.
            temp_data.iloc[i+j+1]['DLY_QTY'] = missing_off
    except:
        pass

In [ ]:
to_save = {
    'y':y,
    'y_hat':y_hat
}
pd.DataFrame(to_save).to_csv('../data/svr1days.csv')

#### 8 Days Validation

In [ ]:
p = Svr()
original_data, _= p.loadData()

N=8
y = []
y_hat = []
for j in range(original_data.shape[0]):
    # Emulate the next day
    temp_data = original_data.copy()
    try:
        for i in range(N):
            pred = []
            # Load the last available sample
            X = temp_data.values[j+i:i+j+WINDOW+1].reshape(WINDOW+1,-1)
            target = temp_data.values[i+j+1:i+j+WINDOW+2, -1].reshape(WINDOW+1,-1)
            # Scale Data
            x_train, y_train, x_test, y_test = p.scaleData(WINDOW, X, target)
            model = SVR(kernel='rbf',gamma='auto',C=1.0,epsilon=0.1)
            model.fit(x_train, y_train[:,0])
            # Predict the next missing sample of Public Offers
            pred.append(model.predict(x_test))
            missing_off = p.descaleData(np.asarray(pred))[0,0]
            # If i is 7, the missing week has been
            # forecasted, so the original sample is put in y, whereas
            # the forecasted one is the target one (t+1)
            if i == N-1:
                y.append(p.descaleData(np.asarray(y_test))[0,0])
                y_hat.append(missing_off)
            # Replace the real data retrieved for training purpose with
            # the forecasted sample.
            temp_data.iloc[i+j+1]['DLY_QTY'] = missing_off
    except:
        pass

In [ ]:
to_save = {
    'y':y,
    'y_hat':y_hat
}
pd.DataFrame(to_save).to_csv('../data/svr8days.csv')

## 4) Models Comparison

In [ ]:
rcParams.update({'font.size': 12})

### 4.1) 8 Days Comparison

In [ ]:
arima = pd.read_csv('../data/arima8days.csv')
lstm = pd.read_csv('../data/lstm8days.csv')
svr = pd.read_csv('../data/svr8days.csv')

Trim datasets to make them comparable

In [ ]:
arima = arima.drop(988)
lstm = lstm.drop(np.arange(0,46))
svr = svr.drop(np.arange(0,45))

#### Line plots

In [ ]:
plt.plot(lstm.y.values[530:561], label='Observation', marker='o', color='C0', markersize=4)
plt.plot(lstm.y_hat.values[530:561], label='LSTM', marker='*', linestyle='-.', color='C1', markersize=4)
plt.plot(arima.y_hat.values[530:561], label='ARIMA(0,1,0)', marker='^', linestyle='--', color='C2', markersize=4)
plt.plot(svr.y_hat.values[530:561], label='SVR', marker='s', linestyle=':', color='C3', markersize=4)

plt.legend()
plt.xlabel('Days')
plt.ylabel('Daily Quantity [MWh]')
plt.grid('--')
plt.xlim(0)
plt.savefig('../fig/comparison8days.png', transparent=True)
plt.close()

#### Scatter plots

In [ ]:
plt.scatter(lstm.y.values, lstm.y_hat.values, color='C3', s=10, alpha=.8, label='LSTM',marker='o')
plt.scatter(lstm.y.values, arima.y_hat.values, color='C1', s=9, alpha=.8, label='ARIMA(0,1,0)',marker='s')
plt.scatter(lstm.y.values, svr.y_hat.values, color='darkgreen', s=10, alpha=.8, label='SVR',marker='^')
plt.plot(lstm.y.values, lstm.y.values, color='r', linewidth=2)

plt.legend()
plt.xlabel('Observations [MWh]')
plt.ylabel('Predictions [MWh]')
plt.grid('--')
plt.xlim(0)
plt.savefig('../fig/scatter8days.png', transparent=True)
plt.close()

#### ARIMA metrics

In [ ]:
mse = mean_squared_error(lstm.y.values, arima.y_hat.values)
mape = np.mean(np.abs((lstm.y.values - arima.y_hat.values) / lstm.y.values)) * 100
mae = mean_absolute_error(lstm.y.values, arima.y_hat.values)
r2 = r2_score(lstm.y.values, arima.y_hat.values)
print(f'\tMSE: {round(mse,2)}')
print(f'\tMAE: {round(mae,2)}')
print(f'\tMAPE: {round(mape,2)}%')
print(f'\tR2: {round(r2,2)}')

#### LSTM metrics

In [ ]:
mse = mean_squared_error(lstm.y.values, lstm.y_hat.values)
mape = np.mean(np.abs((lstm.y.values - lstm.y_hat.values) / lstm.y.values)) * 100
mae = mean_absolute_error(lstm.y.values, lstm.y_hat.values)
r2 = r2_score(lstm.y.values, lstm.y_hat.values)
print(f'\tMSE: {round(mse,2)}')
print(f'\tMAE: {round(mae,2)}')
print(f'\tMAPE: {round(mape,2)}%')
print(f'\tR2: {round(r2,2)}')

#### SVR metrics

In [ ]:
mse = mean_squared_error(lstm.y.values, svr.y_hat.values)
mape = np.mean(np.abs((lstm.y.values - svr.y_hat.values) / lstm.y.values)) * 100
mae = mean_absolute_error(lstm.y.values, svr.y_hat.values)
r2 = r2_score(lstm.y.values, svr.y_hat.values)
print(f'\tMSE: {round(mse,2)}')
print(f'\tMAE: {round(mae,2)}')
print(f'\tMAPE: {round(mape,2)}%')
print(f'\tR2: {round(r2,2)}')

### 4.2) Next Day Comparison

In [ ]:
arima = pd.read_csv('../data/arima1days.csv')
lstm = pd.read_csv('../data/lstm1days.csv')
svr = pd.read_csv('../data/svr1days.csv')

Trim datasets to make them comparable

In [ ]:
arima = arima.drop(988)
lstm = lstm.drop(np.arange(1041,1048))
lstm = lstm.drop(np.arange(0,53))
svr = svr.drop(np.arange(0,52))

#### Line plots

In [ ]:
plt.plot(lstm.y.values[530:561], label='Observation', marker='o', color='C0', markersize=4)
plt.plot(lstm.y_hat.values[530:561], label='LSTM', marker='*', linestyle='-.', color='C1', markersize=4)
plt.plot(arima.y_hat.values[530:561], label='ARIMA(0,1,0)', marker='^', linestyle='--', color='C2', markersize=4)
plt.plot(svr.y_hat.values[530:561], label='SVR', marker='s', linestyle=':', color='C3', markersize=4)

plt.legend()
plt.xlabel('Days')
plt.ylabel('Daily Quantity [MWh]')
plt.grid('--')
plt.ylim(0, 20000)
plt.xlim(0)
plt.savefig('../fig/comparison1days.png', transparent=True)
plt.close()

#### Scatter plots

In [ ]:
plt.scatter(lstm.y.values, lstm.y_hat.values, color='C3', s=10, alpha=.8, label='LSTM',marker='o')
plt.scatter(lstm.y.values, arima.y_hat.values, color='C1', s=9, alpha=.8, label='ARIMA(0,1,0)',marker='s')
plt.scatter(lstm.y.values, svr.y_hat.values, color='darkgreen', s=10, alpha=.8, label='SVR',marker='^')
plt.plot(lstm.y.values, lstm.y.values, color='r', linewidth=2)

plt.legend()
plt.xlabel('Observations [MWh]')
plt.ylabel('Predictions [MWh]')
plt.grid('--')
plt.xlim(0)
plt.savefig('../fig/scatter1days.png', transparent=True)
plt.close()

#### ARIMA metrics

In [ ]:
mse = mean_squared_error(lstm.y.values, arima.y_hat.values)
mape = np.mean(np.abs((lstm.y.values - arima.y_hat.values) / lstm.y.values)) * 100
mae = mean_absolute_error(lstm.y.values, arima.y_hat.values)
r2 = r2_score(lstm.y.values, arima.y_hat.values)
print(f'\tMSE: {round(mse,2)}')
print(f'\tMAE: {round(mae,2)}')
print(f'\tMAPE: {round(mape,2)}%')
print(f'\tR2: {round(r2,2)}')

#### LSTM metrics

In [ ]:
mse = mean_squared_error(lstm.y.values, lstm.y_hat.values)
mape = np.mean(np.abs((lstm.y.values - lstm.y_hat.values) / lstm.y.values)) * 100
mae = mean_absolute_error(lstm.y.values, lstm.y_hat.values)
r2 = r2_score(lstm.y.values, lstm.y_hat.values)
print(f'\tMSE: {round(mse,2)}')
print(f'\tMAE: {round(mae,2)}')
print(f'\tMAPE: {round(mape,2)}%')
print(f'\tR2: {round(r2,2)}')

#### SVR metrics

In [ ]:
mse = mean_squared_error(lstm.y.values, svr.y_hat.values)
mape = np.mean(np.abs((lstm.y.values - svr.y_hat.values) / lstm.y.values)) * 100
mae = mean_absolute_error(lstm.y.values, svr.y_hat.values)
r2 = r2_score(lstm.y.values, svr.y_hat.values)
print(f'\tMSE: {round(mse,2)}')
print(f'\tMAE: {round(mae,2)}')
print(f'\tMAPE: {round(mape,2)}%')
print(f'\tR2: {round(r2,2)}')

## 5) Distribution of ARIMA Forecasting Error

In [ ]:
client = InfluxDBClient('localhost', 8086, 'root', 'root', 'PublicBids')

Retrieve the active operators. Assume that today is the 2/03/2020.<br>
ARIMA works with 60 days as history, so it starts from the 2/01/2020.<br>
The predicted values are referred to the 3/03/2020.

In [ ]:
TODAY = datetime.strptime('15/03/2020', '%d/%m/%Y')
START = TODAY - relativedelta.relativedelta(days=60)
START = int(datetime.timestamp(START)*1e9)

In [ ]:
OP_LIST = []
for market in ['MGP', 'MI', 'MSD']:
    res = client.query(f"SELECT * FROM demand{market} WHERE time >= {START}").raw
    for val in res['series'][0]['values']:
        if val[3] not in OP_LIST and "'" not in val[3]:
            OP_LIST.append(val[3])

    res = client.query(f"SELECT * FROM supply{market} WHERE time >= {START}").raw
    for val in res['series'][0]['values']:
        if val[3] not in OP_LIST and "'" not in val[3]:
            OP_LIST.append(val[3])

Predict the next day.

In [ ]:
y_hat = []
y = []

TODAY = datetime.strptime('2/03/2020', '%d/%m/%Y')
for op in OP_LIST:
    if "'" not in op:
        arima = Arima(op, TODAY)
        pred, val = arima.predict()
        y_hat += pred
        y += val

Sort the absolute point-wise error and visualize it

In [ ]:
errs = []
for i in range(len(y)):
    errs.append(
        np.abs((y[i] - y_hat[i]))
    )
errs.sort()

In [ ]:
plt.figure(figsize=(9, 7))
plt.plot(errs, label=TODAY.strftime('%d/%m/%Y'))
plt.grid()
plt.legend()
plt.xlabel('Number of predictions')
plt.ylabel('Absolute point-wise error')
plt.savefig('../fig/arimaErr.png', transparent=True)

Merge everything and repeat for some days to obtain a performance overview

In [ ]:
TODAY = datetime.strptime('2/03/2020', '%d/%m/%Y')
plt.figure(figsize=(9, 7))

for i in range(5):
    y_hat = []
    y = []

    date = TODAY + relativedelta.relativedelta(days=i)
    for op in OP_LIST:
        if "'" not in op:
            arima = Arima(op, date)
            pred, val = arima.predict()
            y_hat += pred
            y += val

    errs = []
    for i in range(len(y)):
        errs.append(
            np.abs((y[i] - y_hat[i]))
        )
    errs.sort()

    plt.plot(errs, label=date.strftime('%d/%m/%Y'))
plt.grid()
plt.legend()
plt.xlabel('Number of predictions')
plt.ylabel('Absolute point-wise error')
plt.savefig('../fig/arimaErrComp.png', transparent=True)